# 🏎️ F1 Racing Simulation - Google Colab Edition

**Run professional F1 simulations in your browser!**

This notebook lets you:
- ✅ Test unlimited car setups
- ✅ Train RL agents to find optimal racing lines
- ✅ Compare lap times across different configurations
- ✅ Download and validate against real F1 data
- ✅ All within the 12-hour Colab window!

## ⏱️ Time Estimates (on Colab GPU)

| Task | Time |
|------|------|
| Setup | 5 min |
| Train 1 setup (100k steps) | 15-20 min |
| Train 1 setup (500k steps) | 1-2 hours |
| Test 3 setups (500k each) | 3-6 hours |

**You can easily test 5+ setups in 12 hours!**

---

## 📋 Instructions

1. **Enable GPU**: Runtime → Change runtime type → GPU (T4)
2. **Run all cells** in order (Runtime → Run all)
3. **Edit configuration** in the config cell
4. **Results auto-download** before timeout

Let's get started! 🚀

## 🔧 Setup (Run Once)

In [ ]:
# Clone repository
!git clone https://github.com/yourusername/autonomous-racing.git
%cd autonomous-racing

# Install dependencies
!pip install -q -e .

# Install additional packages for Colab
!pip install -q stable-baselines3[extra] gymnasium pyyaml matplotlib seaborn

print("\n✅ Setup complete!")
print("📊 GPU Available:", end=" ")
!nvidia-smi -L

## ⚙️ Configuration - EDIT THIS!

**Customize your simulation here:**

In [ ]:
# =============================================================================
# CONFIGURATION - Edit parameters below
# =============================================================================

CONFIG = {
    # === CIRCUIT ===
    'circuit': 'silverstone',  # Options: silverstone, monaco, spa

    # === TRAINING ===
    'algorithm': 'sac',        # Options: sac, ppo
    'total_timesteps': 200000, # 200k = ~30-40 min (good for Colab)
                               # 500k = ~1-2 hours (best quality)
                               # 100k = ~15-20 min (quick test)

    # === ADVANCED FEATURES ===
    'use_advanced_tire_model': True,  # Pacejka MF 6.2
    'use_advanced_aero': True,        # CFD aerodynamics
    'domain_randomization': 'light',  # Sim-to-real transfer

    # === CAR SETUPS TO TEST ===
    # Add/remove setups as needed
    'car_setups': [
        {
            'name': 'High_Downforce',
            'tire_compound': 'SOFT',
            'front_wing_angle': 15,
            'rear_wing_angle': 12,
            'ride_height_front': 25,  # mm
            'ride_height_rear': 35,
            'fuel_load': 110,
        },
        {
            'name': 'Balanced',
            'tire_compound': 'MEDIUM',
            'front_wing_angle': 11,
            'rear_wing_angle': 9,
            'ride_height_front': 30,
            'ride_height_rear': 40,
            'fuel_load': 110,
        },
        {
            'name': 'Low_Drag',
            'tire_compound': 'SOFT',
            'front_wing_angle': 8,
            'rear_wing_angle': 6,
            'ride_height_front': 35,
            'ride_height_rear': 45,
            'fuel_load': 30,  # Light for quali
        },
    ],

    # === EVALUATION ===
    'num_evaluation_laps': 3,  # Laps per setup

    # === OUTPUT ===
    'output_dir': 'results',
    'auto_download_results': True,  # Download before timeout
}

print("✅ Configuration loaded!")
print(f"Circuit: {CONFIG['circuit']}")
print(f"Setups to test: {len(CONFIG['car_setups'])}")
print(f"Est. time: {len(CONFIG['car_setups']) * CONFIG['total_timesteps'] / 200000 * 30:.0f}-{len(CONFIG['car_setups']) * CONFIG['total_timesteps'] / 200000 * 40:.0f} min")

## 🚀 Run Simulation

This will:
1. Train RL agents for each setup
2. Evaluate lap times
3. Generate comparison report
4. Create visualizations

**Grab a coffee! ☕** (This takes ~30 min - 2 hours depending on settings)

In [ ]:
import numpy as np
import json
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML
import matplotlib.pyplot as plt

from src.envs.f1_racing_env import F1RacingEnv
from src.physics.tire_model import TireCompound

# Create output directory
output_dir = Path(CONFIG['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)
(output_dir / 'models').mkdir(exist_ok=True)
(output_dir / 'telemetry').mkdir(exist_ok=True)
(output_dir / 'visualizations').mkdir(exist_ok=True)

print(f"\n{'='*70}")
print(f"🏎️  F1 RACING SIMULATION - COLAB EDITION")
print(f"{'='*70}")
print(f"Circuit: {CONFIG['circuit'].upper()}")
print(f"Algorithm: {CONFIG['algorithm'].upper()}")
print(f"Training steps: {CONFIG['total_timesteps']:,}")
print(f"Setups to test: {len(CONFIG['car_setups'])}")
print(f"{'='*70}\n")

# Test each setup
results = []

for i, setup in enumerate(CONFIG['car_setups']):
    setup_name = setup['name']

    print(f"\n{'='*70}")
    print(f"🔧 Setup {i+1}/{len(CONFIG['car_setups'])}: {setup_name}")
    print(f"{'='*70}\n")

    # Convert tire compound
    compound_map = {
        'C1': TireCompound.C1, 'C2': TireCompound.C2, 'C3': TireCompound.C3,
        'C4': TireCompound.C4, 'C5': TireCompound.C5,
        'SOFT': TireCompound.C5, 'MEDIUM': TireCompound.C3, 'HARD': TireCompound.C1,
        'INTER': TireCompound.INTERMEDIATE, 'WET': TireCompound.WET,
    }
    tire_compound = compound_map.get(setup.get('tire_compound', 'C3'), TireCompound.C3)

    # Create environment
    env = F1RacingEnv(
        circuit_name=CONFIG['circuit'],
        tire_compound=tire_compound,
        use_advanced_tire_model=CONFIG['use_advanced_tire_model'],
        use_advanced_aero=CONFIG['use_advanced_aero'],
        domain_randomization=CONFIG.get('domain_randomization'),
    )

    # Train model
    print(f"🚀 Training {CONFIG['algorithm'].upper()} agent...\n")

    if CONFIG['algorithm'] == 'sac':
        from src.algorithms.sac_adaptive import train_sac
        model = train_sac(
            env=env,
            total_timesteps=CONFIG['total_timesteps'],
            save_path=str(output_dir / 'models' / f"{setup_name}_sac.zip")
        )
    elif CONFIG['algorithm'] == 'ppo':
        from src.algorithms.ppo_lstm import train_ppo
        model = train_ppo(
            env=env,
            total_timesteps=CONFIG['total_timesteps'],
            save_path=str(output_dir / 'models' / f"{setup_name}_ppo.zip")
        )

    print(f"\n✅ Training complete!\n")

    # Evaluate
    print(f"📊 Evaluating {CONFIG['num_evaluation_laps']} laps...\n")

    best_lap_time = float('inf')
    best_telemetry = []
    all_lap_times = []

    for lap_num in range(CONFIG['num_evaluation_laps']):
        obs, _ = env.reset()
        lap_telemetry = []
        done = False
        truncated = False
        step_count = 0

        while not done and not truncated and step_count < 10000:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)

            lap_telemetry.append({
                'time': info.get('time', step_count * env.dt),
                'distance': info.get('distance', 0.0),
                'speed': info.get('speed', 0.0),
                'throttle': float(action[0]) * 100,
                'brake': float(action[1]) * 100,
            })

            step_count += 1

            if done or truncated:
                lap_time = info.get('lap_time', info.get('time', 0.0))
                all_lap_times.append(lap_time)

                if lap_time < best_lap_time and lap_time > 0:
                    best_lap_time = lap_time
                    best_telemetry = lap_telemetry

                print(f"  Lap {lap_num + 1}: {lap_time:.3f}s")
                break

    # Save telemetry
    telemetry_path = output_dir / 'telemetry' / f"{setup_name}_telemetry.json"
    with open(telemetry_path, 'w') as f:
        json.dump({
            'setup': setup,
            'best_lap_time': best_lap_time,
            'all_lap_times': all_lap_times,
            'telemetry': best_telemetry,
        }, f, indent=2)

    print(f"\n✅ Best lap: {best_lap_time:.3f}s")
    if len(all_lap_times) > 1:
        print(f"   Average: {np.mean(all_lap_times):.3f}s ± {np.std(all_lap_times):.3f}s")

    results.append({
        'setup_name': setup_name,
        'setup_config': setup,
        'best_lap_time': best_lap_time,
        'avg_lap_time': np.mean(all_lap_times) if all_lap_times else best_lap_time,
        'lap_times': all_lap_times,
        'telemetry': best_telemetry,
    })

print(f"\n{'='*70}")
print("✅ ALL SETUPS COMPLETE!")
print(f"{'='*70}\n")

## 📊 Results & Comparison

In [ ]:
# Sort results by lap time
sorted_results = sorted(results, key=lambda x: x['best_lap_time'])

# Print comparison table
print(f"\n{'='*70}")
print(f"📊 SETUP COMPARISON - {CONFIG['circuit'].upper()}")
print(f"{'='*70}\n")

print(f"{'Rank':<8} {'Setup':<20} {'Best Lap':<12} {'Compound':<10}")
print(f"{'-'*70}")

for i, result in enumerate(sorted_results):
    rank = ["🥇", "🥈", "🥉"][i] if i < 3 else f"#{i+1}"
    print(f"{rank:<8} {result['setup_name']:<20} {result['best_lap_time']:.3f}s    {result['setup_config'].get('tire_compound', 'N/A'):<10}")

# Highlight best
best = sorted_results[0]
print(f"\n{'-'*70}")
print(f"🏆 OPTIMAL SETUP: {best['setup_name']}")
print(f"⏱️  LAP TIME: {best['best_lap_time']:.3f}s")

if len(sorted_results) > 1:
    delta = sorted_results[1]['best_lap_time'] - best['best_lap_time']
    print(f"📈 ADVANTAGE: +{delta:.3f}s ({delta/best['best_lap_time']*100:.2f}%)")

print(f"{'='*70}\n")

## 📈 Visualizations

In [ ]:
# Setup comparison bar chart
fig, ax = plt.subplots(figsize=(12, 6))

names = [r['setup_name'] for r in sorted_results]
times = [r['best_lap_time'] for r in sorted_results]
colors = ['gold' if i == 0 else 'steelblue' for i in range(len(names))]

bars = ax.bar(names, times, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_ylabel('Lap Time (s)', fontsize=13, fontweight='bold')
ax.set_title(f'Setup Comparison - {CONFIG["circuit"].title()} Circuit',
             fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{height:.3f}s',
           ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save
viz_path = output_dir / 'visualizations' / 'setup_comparison.png'
plt.savefig(viz_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Saved: {viz_path}")

In [ ]:
# Speed trace comparison (top 3 setups)
if all('telemetry' in r and r['telemetry'] for r in sorted_results[:3]):
    fig, ax = plt.subplots(figsize=(14, 6))

    colors = ['gold', 'silver', '#CD7F32']  # Gold, Silver, Bronze
    markers = ['o', 's', '^']

    for i, result in enumerate(sorted_results[:3]):
        telemetry = result['telemetry']
        distances = [t['distance'] for t in telemetry]
        speeds = [t['speed'] for t in telemetry]

        ax.plot(np.array(distances) / 1000, speeds,
               label=f"{result['setup_name']} ({result['best_lap_time']:.3f}s)",
               linewidth=2.5, alpha=0.8, color=colors[i],
               marker=markers[i], markersize=3, markevery=50)

    ax.set_xlabel('Distance (km)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Speed (km/h)', fontsize=12, fontweight='bold')
    ax.set_title('Speed Trace Comparison - Top 3 Setups',
                fontsize=14, fontweight='bold', pad=15)
    ax.legend(loc='best', fontsize=10, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')

    plt.tight_layout()

    # Save
    speed_path = output_dir / 'visualizations' / 'speed_comparison.png'
    plt.savefig(speed_path, dpi=150, bbox_inches='tight')
    plt.show()

    print(f"✅ Saved: {speed_path}")
else:
    print("⚠️ No telemetry data available for speed trace comparison")

## 💾 Download Results

**Download all results before Colab session ends!**

In [ ]:
# Create archive of all results
import shutil
from google.colab import files

# Create zip archive
archive_name = f"f1_results_{CONFIG['circuit']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
shutil.make_archive(archive_name, 'zip', output_dir)

print(f"\n📦 Created archive: {archive_name}.zip")
print(f"Size: {Path(f'{archive_name}.zip').stat().st_size / 1024 / 1024:.1f} MB")
print(f"\nDownloading...")

# Download
files.download(f"{archive_name}.zip")

print(f"\n✅ Download complete!")
print(f"\nArchive contains:")
print(f"  - Trained models (.zip)")
print(f"  - Telemetry data (.json)")
print(f"  - Visualizations (.png)")
print(f"\nExtract the archive on your computer to view results!")

## 🎯 Optional: Download Real F1 Data

Download official F1 telemetry for validation!

In [ ]:
# Install FastF1
!pip install -q fastf1

import fastf1
import pandas as pd

# Download Hamilton's Silverstone qualifying lap
year = 2023
circuit = 'Silverstone'
session_type = 'Q'
driver = 'HAM'

print(f"Downloading {driver}'s {circuit} {session_type} lap from {year}...\n")

session = fastf1.get_session(year, circuit, session_type)
session.load()

laps = session.laps.pick_driver(driver)
fastest = laps.pick_fastest()
telemetry = fastest.get_telemetry()

# Export
export_data = pd.DataFrame({
    'time': telemetry['Time'].dt.total_seconds() - telemetry['Time'].dt.total_seconds().iloc[0],
    'distance': telemetry['Distance'].values,
    'speed': telemetry['Speed'].values,
    'throttle': telemetry['Throttle'].values,
    'brake': telemetry['Brake'].values,
    'gear': telemetry['nGear'].values,
})

Path('data/real_f1').mkdir(parents=True, exist_ok=True)
export_data.to_csv(f'data/real_f1/{circuit.lower()}_{driver}_{year}.csv', index=False)

print(f"\n✅ Downloaded real F1 data!")
print(f"Lap time: {fastest['LapTime']}")
print(f"Max speed: {export_data['speed'].max():.1f} km/h")
print(f"\nYou can now validate your simulation against this data!")

## 📝 Summary

**What you accomplished:**

✅ Tested multiple F1 car setups  
✅ Trained RL agents to find optimal driving  
✅ Compared lap times across configurations  
✅ Generated professional visualizations  
✅ Downloaded all results for offline analysis  

**Next steps:**

1. Extract downloaded zip file
2. Review visualizations and telemetry
3. Modify configuration and run again
4. Compare with real F1 data

**Want to run again?**

- Edit the Configuration cell
- Runtime → Restart and run all

---

**Questions or issues?**

Check the [QUICKSTART.md](https://github.com/yourusername/autonomous-racing/blob/main/QUICKSTART.md) or [documentation](https://github.com/yourusername/autonomous-racing).

Happy racing! 🏎️💨